In [1]:
import numpy as np 
import time 
from serial import Serial 
import re 
from numpy import array 
from random import randint, randrange, random

In [ ]:
port = "COM6"
board = serial.Serial(port,19200,timeout=5)
time.sleep(3.0)

In [ ]:
pop = 0.95
paused= False
hitwall = False
def save_synapses(syn0_arry, syn1_arry, syn2_arry, syn3_arry,i):
    np.save('syn0_arry'+str(i)+'.npy',syn0_arry)
    np.save('syn1_arry'+str(i)+'.npy',syn1_arry)
    np.save('syn2_arry'+str(i)+'.npy',syn2_arry)
    np.save('syn3_arry'+str(i)+'.npy',syn3_arry)
def nonlin(x):
    return 1/(1+np.exp(-x))
def paused_true():
    global paused
    paused = True
def paused_false():
    global paused
    paused = False
def hitwall_true():
    global hitwall
    hitwall = True
def hitwall_false():
    global hitwall
    hitwall = False
def get_data_input():
    board.write(b"g")
    time.sleep(4.1)
    data_input = []
    while board.in_waiting:
        input_data = board.readline()
        x= str(input_data.decode())
        x = re.sub("[^\d\.]", "", x)
        print(x)
        if '.' in x:
            data_input.append(float(x))
        elif '677' in x:
           
            hitwall_true()
            print('HIT THE WALL!!!')
            break
        elif '788' in x:
           
            paused_true()
            print('TRAINING PAUSED')
            break
        else:
            data_input.append(int(x))
    return data_input

def encoded_input():
    data_input = get_data_input()
    input_layer = np.zeros((17,250))
    for i in range(len(data_input)):
        if data_input[i]>250:
            data_input[i] = 250
        input_layer[i,int(data_input[i]-1)]= 1
    return input_layer
def very_forward():
    board.write(b"a")
def forward():
    board.write(b"b")
def very_left():
    board.write(b"c")
def left():
    board.write(b"d")
def very_right():
    board.write(b"e")
def right():
    board.write(b"f")


def zero_or_one(arry):
    results = []
    max_num = 0
    max_index = 0
    z = arry[0]
    for i in range(len(z)):
        if z[i]>max_num:
            max_num = z[i]
            max_index = i
    for l in range(len(z)):
        z[l] = 0
    z[max_index] = 1
    for i in z:
        results.append(i)
        
    print(results)
    return results

def react_to_results(results):
    time_slept = 0
    forward_metric = 0
    trigger=0
    for i in range(len(results)):
                    if i == 0:
                        if results[i]==1:
                            very_forward()
                            forward_metric+=2
                    elif i ==1:
                        if results[i]==1:
                            forward()
                            forward_metric+=1
                    elif i ==2:
                        if results[i]==1:
                            very_left()
                    elif i ==3:
                        if results[i]==1:
                            left()
                    elif i ==4:
                        if results[i]==1:
                            very_right()
                    elif i ==5:
                        if results[i]==1:
                            right()
                    time.sleep(0.5)
                    time_slept += 0.5
                    global hitwall
                    global paused
                    if hitwall == True:
                        trigger=1
                        break
                    if paused == True:
                        trigger=2
                        break
    return time_slept,forward_metric,trigger

def neural_network_results(syn0_arry, syn1_arry, syn2_arry, syn3_arry):
    l0 = encoded_input()
    l1 = nonlin(np.dot(l0,syn0_arry))
    l2 = nonlin(np.dot(l1,syn1_arry))
    l3 = nonlin(np.dot(l2,syn2_arry))
    l4 = nonlin(np.dot(syn3_arry,l3))
    results=zero_or_one(l4)
    return results
def load_synapses(i):
    syn0_arry = np.load('syn0_arry'+str(i)+'.npy')
    syn1_arry = np.load('syn1_arry'+str(i)+'.npy')
    syn2_arry = np.load('syn2_arry'+str(i)+'.npy')
    syn3_arry = np.load('syn3_arry'+str(i)+'.npy')
    return syn0_arry, syn1_arry, syn2_arry, syn3_arry

def get_top2_fitness(arry):
    top_one = 0
    top_two = 0
    top_one_index = 0
    top_two_index = 0
    for i in range(len(arry)):
        if arry[i] >= top_one:
            top_two = top_one
            top_two_index = top_one_index
            top_one = arry[i]
            top_one_index = i
        elif  arry[i]> top_two:
            top_two = arry[i]
            top_two_index = i
    return_array = [top_one,top_two]
    index_arry = [top_one_index,top_two_index]
    return return_array, index_arry

def mutate_top_synapses(syn0_arry, syn1_arry, syn2_arry, syn3_arry):
    global paused
    global pop 
    
    if pop<0.95:
        pop+=0.05
    
    if paused == True:
        return syn0_arry, syn1_arry, syn2_arry, syn3_arry
    syn0_arry1 =[]
    syn1_arry1 =[]
    syn2_arry1 = []
    syn3_arry1 = []
    for l in range(10):
        main = syn0_arry
        z = randint(0,1)
        for x in range(250):
            for y in range(17):
                if random() < 1-pop:
                     main[z][x,y] = randrange(-1,1)
        syn0_arry1.append(main)
    for l in range(10):
        main = syn1_arry
        z = randint(0,1)
        for x in range(17):
            for y in range(10):
                if random() < 1-pop:
                    main[z][x,y] = randrange(-1,1)
        syn1_arry1.append(main)
    for l in range(10):
        main = syn2_arry
        z = randint(0,1)
        for x in range(10):
            for y in range(6):
                if random() < 1-pop:
                    main[z][x,y] = randrange(-1,1)
        syn2_arry1.append(main)
    for l in range(10):
        main = syn3_arry
        z = randint(0,1)
        for x in range(1):
            for y in range(17):
                if random() < 1-pop:
                    main[z][x,y] = randrange(-1,1) 
        syn3_arry1.append(main)
    return syn0_arry1, syn1_arry1, syn2_arry1, syn3_arry1

def fitness(forward_metric, time_metric):
    results = (forward_metric-time_metric)/8000
    return results


def best_fitness_synapse_sets(syn0_arry, syn1_arry, syn2_arry, syn3_arry, fitness_array):
    global paused
    if paused == True:
        return syn0_arry, syn1_arry, syn2_arry, syn3_arry
    
    fitness_array1,indexes_array = get_top2_fitness(fitness_array)
        
    syn0_arry1 = [syn0_arry[indexes_array[0]], syn0_arry[indexes_array[1]]]
    syn1_arry1 = [syn1_arry[indexes_array[0]] , syn1_arry[indexes_array[1]]]
    syn2_arry1 = [syn2_arry[indexes_array[0]], syn2_arry[indexes_array[1]]]
    syn3_arry1 = [syn3_arry[indexes_array[0]] , syn3_arry[indexes_array[1]]]         
    return syn0_arry1, syn1_arry1, syn2_arry1, syn3_arry1 

def start_training():
    global paused
    # choose same p as i in np.load
    p=0  
    syn0_arry, syn1_arry, syn2_arry, syn3_arry = load_synapses(p)
    if paused == True:
        return
    global hitwill
    while paused==False:
        p+=1
        fitness_array = []
        for i in range(10):
            fitness1 = []
            hitwall = False
            results = neural_network_results(syn0_arry[i], syn1_arry[i], syn2_arry[i], syn3_arry[i])
            time_metric = 0
            forward_metric = 0
            time_start = time.time()
            time_slept = 0
            l=0
            trigger = 0
            hitwall_false()
            while trigger==0:
                if l==0:
                    time_slept1,forward_metric1,trigger = react_to_results(results)
                    l+=1
                else:
                    results = neural_network_results(syn0_arry[i], syn1_arry[i], syn2_arry[i], syn3_arry[i])
                    time_slept1,forward_metric1,trigger = react_to_results(results)
                
                time_slept +=time_slept1
                forward_metric += forward_metric1
            time_metric = time.time() - time_start - time_slept
            fitness1 = fitness(forward_metric, time_metric)
            fitness_array.append(fitness)
            if paused==True:
                break
        if(paused==True):
            return
        syn0_arry, syn1_arry, syn2_arry, syn3_arry = mutate_top_synapses(best_fitness_synapse_sets(syn0_arry, syn1_arry, syn2_arry, syn3_arry))
       
        save_synapses(syn0_arry, syn1_arry, syn2_arry, syn3_arry,p)